# This script will used OSCA (OmicS-data-based Complex trait Analysis) for analysis of variance explained and mixed linear models 

Author: Jose Jaime Martinez-Magana

Day: 15 February 2023

This script will use OSCA to estimate variance explained and mixed linear models in the analysis of SES in children.
OSCA is found in https://yanglab.westlake.edu.cn/software/osca/#Overview

In [ ]:
# path to OSCA folder
# /gpfs/gibbs/project/montalvo-ortiz/jjm262/programs/osca
# this path should be change to your specific path
# downloading OSCA
wget https://yanglab.westlake.edu.cn/software/osca/download/osca-0.46.1-linux-x86_64.zip
# unzip the file
unzip osca-0.46.1-linux-x86_64.zip
# remove zip file
rm osca-0.46.1-linux-x86_64.zip

In [ ]:
# first we need to create an annotation matrix fallowing the structure of annotation
#chr   probeID    pos  probeID  strand

# request computational resources
srun --pty --mem=8G -p interactive bash
# load environment
module load miniconda
# activate environment
conda activate ewas_saliva

# openning R
R
# loading annotation data
library(IlluminaHumanMethylation450kanno.ilmn12.hg19)
data(IlluminaHumanMethylation450kanno.ilmn12.hg19)

# setting output path
outp="/vast/palmer/scratch/montalvo-ortiz/jjm262/epigenomics/ewas_saliva_ses/databases/annot/annot_file.txt"

# get illunina annotations
kanno=IlluminaHumanMethylation450kanno.ilmn12.hg19::Locations
# remove "chr" from chromosome
kanno$chr=gsub("chr","",kanno$chr)
# change X code to 23 and Y to 24
kanno$chr[kanno$chr == "X"] = 23
kanno$chr[kanno$chr == "Y"] = 24
# transforming position and chromosome to numeric
kanno$pos=as.numeric(kanno$pos)
kanno$chr=as.numeric(kanno$chr)
# ordering by chromosome and position
kanno=kanno[order(kanno$chr, kanno$pos),]
# changing the annotation structure of the dataframe
kanno2=data.frame(chr=kanno$chr, probeID=rownames(kanno), pos=kanno$pos, probeID2=rownames(kanno), strand=kanno$strand)
# write annotation dataframe
write.table(file=outp,
           kanno2,
           row.names=FALSE,
           sep="\t",
           quote=FALSE)
# exit R
q()

# deactivate environment
conda deactivate

# exit computaional resources
exit

In [ ]:
# request computational resources
# we are going to use a script that requieres multithreads to run change the --threads parameter for speeding
srun --pty --mem=32G --threads=4 -p interactive bash
# load environment
module load miniconda
# activate environment
conda activate ewas_saliva

####################################################################################
# we developed a script for extracting the beta matrix, covariates, and phenofiles 
# from the qcdata for saliva samples and transform to osca format
# the script is called qcdata_to_osca.Rscript
#################################################################################################
# content of the script

#!/usr/bin/env Rscript --vanilla --slave
##################################################################################################
# Rscript to generate input files for osca, using the qc'ed file 
# day: 15 February 2023
# author: Jose Jaime Martinez-Magana
####################################################################################
# This script uses three inputs the qcdata following this github https://github.com/martinezjaime/ewas_saliva_ses/blob/main/qc_data/probe_and_sample_quality_control.ipynb,
# a list of samples to be included in the analysis as csv file

library(optparse) 
option_list=list(
    make_option(c("--file"),
                type="character",
                default=NULL,
                help="path to *rds object with qc information", metavar="character"),
    make_option(c("--out"),
                type="character",
                default="out.rds",
                help="output file name for rds [default= %default]", metavar="character"),
    make_option(c("--outname"),
                type="character",
                default=NULL,
                help="name of the rds file after performing this analysis", metavar="character"),
    make_option(c("--samplelist"),
                type="character",
                default=NULL,
                help="path to a *csv file with a list of samples to subset the analysis. This script uses a file with SampleID with Array_Sentrix structures and a header with SampleID", metavar="character"),
    make_option(c("--pheno"),
                type="character",
                default=NULL,
                help="name of the column identifying phenotype to be tested in the regression model", metavar="character"),
    make_option(c("--covar"),
                type="character",
                default=NULL,
                help="list of covariates to be included in the analysis from the phenofile separeted by ','", metavar="character")
);

opt_parser=OptionParser(option_list=option_list);
opt=parse_args(opt_parser);

if (is.null(opt$file)){
  print_help(opt_parser)
  stop("At least one argument must be supplied (input file)", call.=FALSE)
}

########################################################################################################################
# this part of the script will be removed from the script
# is only for testing the script
# create a empty list to simulate the options in optparse library
opt=list()
opt$file="/vast/palmer/scratch/montalvo-ortiz/jjm262/epigenomics/ewas_saliva_ses/databases/qced/qced_data_v02062023.rds"
opt$out="/vast/palmer/scratch/montalvo-ortiz/jjm262/epigenomics/ewas_saliva_ses/databases/qced/osca"
opt$outname="beta_gcta_v02152023"
opt$pheno=c("SES")
opt$covar=c('age,gender')
########################################################################################################################

# load minfi for package compatibility
library(minfi)
library(data.table)

# loading rds
paste0("Start analysis of data:",Sys.time(),"---","###Analysis path[",opt$file,"]###")
file=readRDS(opt$file)
# setting output file
outfile=paste0(opt$out,opt$outname,sep="")
paste0("Input files for gcta will be saved to:",Sys.time(),"---","###Output path[",outfile,"_*]###")

# loading phenotype file
paste0("Loading pheno file:",Sys.time())
# add pheno to be tested
pheno=opt$pheno
# add covariates to be tested
covar=unlist(strsplit(opt$covar,','))
# get covars from pheno file
covars=c(pheno,covar)
covars=c(pheno,covar,"SampleID")
# select covars from phenofile
pheno_data=file$pheno[,colnames(file$pheno) %in% covars]
# add SampleID as rownames
rownames(pheno_data)=pheno_data$SampleID
# remove SampleID column
pheno_data$SampleID=NULL

# get cells
paste0("Adding cells to pheno file:",Sys.time())
cells=file$cell_epidish
# transform cells to data frame
cells2=as.data.frame(cells)
# remove unneccesary object
rm(cells)
# removing cells with cero numbers in all samples
cells3=cells2[,colSums(cells2) > 0.00]
paste0("Warning: This script merges CD4T + CD8T cells to Limphocytes imputed with EpiDISH from saliva, please revised the cell distribution in your data")
# merging CD4T and CD8T cells
cells3_m=cells3[,"CD4T", drop = F] + cells3[,"CD8T", drop = F]
colnames(cells3_m)="Lympho"
cells3_m2=cbind(cells3_m, cells3[,c("Epi","Fib","B","NK","Mono","Neutro")])
# add warning to say the user the cells that will be used in the analysis
paste0("Using the following cells in the regression models:",colnames(cells3_m2))
# merge pheno_data with cells
pheno_data=cbind(pheno_data,cells3_m2)

# subset samples if sample list is added
# subset phenofile
# here we used is.null, to ask if there was a sample list added to the script
if(!is.null(opt$samplelist)){
    paste0("Sample filter added, phenofile will be filtered based in:",opt$samplelist,"---",Sys.time())
    sample_f=read.csv(opt$samplelist)
    # subsetting pheno file
    pheno_data=pheno_data[rownames(pheno_data) %in% sample_f$SampleID,]
} else {
    pheno_data=pheno_data
}

# subset beta matrix
if(!is.null(opt$samplelist)){
    paste0("Sample filter added, beta matrix will be filtered based in:",opt$samplelist,"---",Sys.time())
    sample_f=read.csv(opt$samplelist)
    bvalues=file$rgsetraw_bmiq
    bvalues=bvalues[,colnames(bvalues) %in% sample_f$SampleID]
} else {
    bvalues=file$rgsetraw_bmiq
}

# sample sanity check
all_samples=is.null(pheno_data[!rownames(pheno_data) %in% colnames(bvalues),])
if(all_samples == TRUE){
    paste0("All samples in the phenofile found in the beta value matrix")
    bvalues=bvalues
} else {
    missing_samples=rownames(pheno_data[!rownames(pheno_data) %in% colnames(bvalues),])
    pheno_data=pheno_data[!rownames(pheno_data) %in% missing_samples,]
    bvalues=bvalues[, colnames(bvalues) %in% rownames(pheno_data)]
    paste0("The following samples were not found in the beta values but found in the pheno file: ", missing_samples)
}

# ordering pheno file based on beta
paste0("Warning: this script orders the pheno file based on the beta matrix:", Sys.time())
pheno_data=pheno_data[match(rownames(pheno_data),colnames(bvalues)),]

# transposing bvalues
bvalues_t=t(bvalues)

# building beta values structures
fid_iid=data.frame(FID=rownames(bvalues_t),IID=rownames(bvalues_t))
# adding fid_iid to bvalues matrices
bvalues_t=cbind(fid_iid,bvalues_t)
# saving the txt beta matrices for osca
# making object for naming beta matrix
betaout=paste0(outfile,"_datafile_beta.txt",sep="")
paste0("Start writting beta matrix", Sys.time())
# using data.table to optimized writting
data.table::fwrite(bvalues_t, file=betaout, row.names=FALSE, quote=FALSE, sep="\t")

# building phenofile structures
pheno_f=pheno_data[opt$pheno]
# adding fid_iid to phenofile
pheno_f=cbind(fid_iid,pheno_f)
# saving phenofile for osca
# making object for output name of the pheno file
phenoout=paste0(outfile,"_datafile_pheno.txt",sep="")
paste0("Start writting phenofile: ", Sys.time())
write.table(pheno_f, file=phenoout, row.names=FALSE, quote=FALSE, sep="\t")

# building covarfile structures
covar_f=pheno_data[!colnames(pheno_data) %in% opt$pheno]
# adding fid_iid to phenofile
covar_f=cbind(fid_iid,covar_f)
# saving covar for osca
# making object for output name of the covar file
covarout=paste0(outfile,"_datafile_covar.txt",sep="")
paste0("Start writting covarfile: ", Sys.time())
write.table(covar_f, file=covarout, row.names=FALSE, quote=FALSE, sep="\t")

In [ ]:
# executing the Rscript to transform data to glint format
# move to the directory storing the Rscript
Rscript qcdata_to_osca.Rscript --file=/vast/palmer/scratch/montalvo-ortiz/jjm262/epigenomics/ewas_saliva_ses/databases/qced/qced_data_v02062023.rds \
--out=/vast/palmer/scratch/montalvo-ortiz/jjm262/epigenomics/ewas_saliva_ses/databases/qced/osca/ \
--outname=qced_data_v02152023_osca \
--pheno=SES \
--covar=age,gender

# deactivate environment
conda deactivate

In [ ]:
# making osca files
# setting osca path
osca_p="/gpfs/gibbs/project/montalvo-ortiz/jjm262/programs/osca/osca-0.46.1-linux-x86_64/osca-0.46.1"
# setting beta matrix path
beta_p="/vast/palmer/scratch/montalvo-ortiz/jjm262/epigenomics/ewas_saliva_ses/databases/qced/osca/qced_data_v02152023_osca_datafile_beta.txt"
# setting output
out_p="/vast/palmer/scratch/montalvo-ortiz/jjm262/epigenomics/ewas_saliva_ses/databases/qced/osca/qced_data_v02152023_osca_files"

# running osca to build the osca files
${osca_p} --efile ${beta_p} --methylation-beta --make-bod --out ${out_p}

# updating annotation information
# set annotation file
annot_f="/vast/palmer/scratch/montalvo-ortiz/jjm262/epigenomics/ewas_saliva_ses/databases/annot/annot_file.txt"
${osca_p} --befile ${out_p} --update-opi ${annot_f}

# creating ORM
# setting ORM output
out_orm="/vast/palmer/scratch/montalvo-ortiz/jjm262/epigenomics/ewas_saliva_ses/databases/qced/osca/qced_data_v02152023_osca_orm"
${osca_p} --befile ${out_p} --make-orm --out ${out_orm}

# splitting covar file in discrete and quantative covariates
cd /vast/palmer/scratch/montalvo-ortiz/jjm262/epigenomics/ewas_saliva_ses/databases/qced/osca
# cutting discrete covars
cut -f 1-2,4 qced_data_v02152023_osca_datafile_covar.txt > qced_data_v02152023_osca_datafile_discovar.txt
# cutting continuos covars
cut --complement -f 4 qced_data_v02152023_osca_datafile_covar.txt > qced_data_v02152023_osca_datafile_contcovar.txt

# running GREML
${osca_p} --reml --orm ${out_orm} \
--pheno /vast/palmer/scratch/montalvo-ortiz/jjm262/epigenomics/ewas_saliva_ses/databases/qced/osca/qced_data_v02152023_osca_datafile_pheno.txt \
--covar /vast/palmer/scratch/montalvo-ortiz/jjm262/epigenomics/ewas_saliva_ses/databases/qced/osca/qced_data_v02152023_osca_datafile_discovar.txt \
--qcovar /vast/palmer/scratch/montalvo-ortiz/jjm262/epigenomics/ewas_saliva_ses/databases/qced/osca/qced_data_v02152023_osca_datafile_contcovar.txt \
--out /vast/palmer/scratch/montalvo-ortiz/jjm262/epigenomics/ewas_saliva_ses/results/assoc/mixed_models_osca/reml_all_samples

# in this analysis we have an error
# Performing  REML analysis ...
# 134 observations, 10 fixed effect(s), and 2 variance component(s)(including residual variance).
# Calculating prior values of variance components by EM-REML ...
# Updated prior values: 735.682459        2136.204405
# logL: -4931.300
# Running AI-REML algorithm ...
# Iter.   logL    V(O)    V(e)
# Error: matrix H is not invertible.

# running MOMENT model in OSCA
${osca_p} --moment2-beta \
--befile ${out_p} \
--pheno /vast/palmer/scratch/montalvo-ortiz/jjm262/epigenomics/ewas_saliva_ses/databases/qced/osca/qced_data_v02152023_osca_datafile_pheno.txt \
--covar /vast/palmer/scratch/montalvo-ortiz/jjm262/epigenomics/ewas_saliva_ses/databases/qced/osca/qced_data_v02152023_osca_datafile_discovar.txt \
--moment-cor \
--cor-r2 0.6 \
--out /vast/palmer/scratch/montalvo-ortiz/jjm262/epigenomics/ewas_saliva_ses/results/assoc/mixed_models_osca/moment_all_samples_v02162023
